# Phase 3: The Mathematical Engine

This notebook demonstrates the mathematical foundations of the Coastal Labor-Resilience Engine:

1. **Markov Transition Matrix** - Labor state dynamics with climate shock modifiers
2. **Resilience ODE** - $\frac{dL}{dt} = rL(1 - \frac{L}{K}) - \beta(EJ)$
3. **Coupled Model** - Combined Markov + ODE simulation

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

# Phase 3 imports
from src.models.markov_chain import (
    MarkovTransitionMatrix,
    LaborState,
    ShockParameters,
    STATE_INDEX,
    create_regional_chain
)
from src.models.resilience_ode import (
    ResilienceODE,
    ODEParameters,
    EJScreenData,
    ClimateShock,
    CoupledMarkovODE,
    create_shock_scenario,
    create_ej_tract
)

# Plotting config
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Phase 3 imports successful!")

## Part 1: Markov Transition Matrix

### Labor States
- **COASTAL**: Climate-sensitive jobs (fishing, tourism, outdoor work)
- **INLAND**: Climate-resilient jobs (remote work, tech, healthcare)
- **UNEMPLOYED**: Out of workforce
- **TRANSITIONING**: Active job search/retraining

In [ ]:
# Create Markov chain
markov = MarkovTransitionMatrix()
print(markov)

# View baseline transition matrix
print("\n📊 Baseline Transition Matrix (P_ij = prob from state i to j):")
print("States: COASTAL, INLAND, UNEMPLOYED, TRANSITIONING")
print(markov.baseline.round(3))

In [ ]:
# Compute steady-state distribution
steady = markov.compute_steady_state()

print("📈 Steady-State Distribution (long-term equilibrium):")
for state in LaborState:
    idx = STATE_INDEX[state]
    print(f"  {state.value:15s}: {steady[idx]:.1%}")

In [ ]:
# Analyze full chain properties
analysis = markov.full_analysis()

print("🔬 Markov Chain Analysis:")
print(f"  Ergodic: {analysis.is_ergodic}")
print(f"  Entropy Rate: {analysis.entropy_rate:.3f} bits")
print(f"  Mixing Time: {analysis.mixing_time} steps to equilibrium")
print(f"\n  Eigenvalues: {np.round(analysis.eigenvalues.real, 3)}")

### Climate Shock Impact on Transitions

In [ ]:
# Define a climate shock (e.g., major storm)
shock = ShockParameters(
    severity=0.7,           # 70% severity
    duration_days=14,
    affected_industries=["fishing", "tourism", "outdoor_recreation"]
)

# Compare baseline vs shock
impact = markov.compare_shock_impact(shock)

print("🌊 Climate Shock Impact Analysis:")
print(f"\n  Severity: {shock.severity:.0%}")
print(f"  Duration: {shock.duration_days} days")

print("\n  Steady-State Shifts:")
for i, state in enumerate(LaborState):
    shift = impact['impact']['steady_state_shift'][i]
    direction = "↑" if shift > 0 else "↓"
    print(f"    {state.value:15s}: {direction} {abs(shift):.1%}")

print(f"\n  Entropy Change: {impact['impact']['entropy_change']:+.3f} bits")
print(f"  Mixing Time Change: {impact['impact']['mixing_time_change']:+d} steps")

### Simulate Population Dynamics

In [ ]:
# Initial population distribution (typical Santa Barbara)
initial_dist = np.array([0.35, 0.50, 0.08, 0.07])  # 35% coastal, 50% inland, 8% unemployed, 7% transitioning

# Simulate 180 days with shock at day 30
n_days = 180
shock_schedule = {30: ShockParameters(severity=0.6, duration_days=14)}

history = markov.simulate_population(
    initial_distribution=initial_dist,
    n_steps=n_days,
    shock_schedule=shock_schedule
)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

days = np.arange(n_days + 1)
colors = ['#e74c3c', '#27ae60', '#f39c12', '#3498db']
labels = ['Coastal', 'Inland', 'Unemployed', 'Transitioning']

for i, (color, label) in enumerate(zip(colors, labels)):
    ax.plot(days, history[:, i] * 100, color=color, linewidth=2, label=label)

# Mark shock period
ax.axvspan(30, 44, alpha=0.2, color='red', label='Climate Shock')
ax.axvline(30, color='red', linestyle='--', alpha=0.5)

ax.set_xlabel('Days', fontsize=12)
ax.set_ylabel('Population %', fontsize=12)
ax.set_title('Labor Market Dynamics During Climate Shock', fontsize=14)
ax.legend(loc='right')
ax.set_xlim(0, n_days)

plt.tight_layout()
plt.show()

## Part 2: Resilience ODE

The **Labor Resilience ODE**:

$$\frac{dL}{dt} = rL\left(1 - \frac{L}{K}\right) - \beta(EJ)$$

Where:
- $L(t)$: Labor force participation at time $t$
- $r$: Recovery rate (job creation speed)
- $K$: Carrying capacity (max sustainable employment)
- $\beta(EJ)$: **Environmental Justice friction** - derived from EPA EJScreen data

In [ ]:
# Create EJScreen profiles for different neighborhoods
low_burden = create_ej_tract("low_burden", "060830001001")
average = create_ej_tract("average", "060830002001")
high_burden = create_ej_tract("high_burden", "060830003001")
extreme_burden = create_ej_tract("extreme_burden", "060830004001")

print("📊 EJScreen Friction Coefficients (β):")
print(f"  Low Burden:     β = {low_burden.compute_beta():.3f}")
print(f"  Average:        β = {average.compute_beta():.3f}")
print(f"  High Burden:    β = {high_burden.compute_beta():.3f}")
print(f"  Extreme Burden: β = {extreme_burden.compute_beta():.3f}")

print("\n  Higher β = more friction = slower labor market recovery")

In [ ]:
# Create ODE solver with different EJ profiles
params = ODEParameters(r=0.1, K=1.0, beta=0.05)
ode = ResilienceODE(params)

# Add a climate shock
shock = create_shock_scenario("major_hurricane", start_time=30)
ode.add_shock(shock)

print(f"🌀 Climate Shock: {shock.severity:.0%} severity, {shock.duration} day duration")
print(f"   Carrying capacity reduction: {shock.K_reduction:.0%}")
print(f"   Friction increase: +{shock.beta_increase:.2f}")

In [ ]:
# Compare trajectories across EJ profiles
tracts = [low_burden, average, high_burden, extreme_burden]
tract_names = ["Low Burden", "Average", "High Burden", "Extreme Burden"]

solutions = ode.compare_tracts(
    L0=0.92,           # 92% employment
    t_span=(0, 365),   # 1 year simulation
    tracts=tracts,
    n_points=365
)

# Plot trajectories
fig, ax = plt.subplots(figsize=(12, 7))

colors = ['#2ecc71', '#3498db', '#e67e22', '#c0392b']
for (tract_id, sol), name, color in zip(solutions.items(), tract_names, colors):
    ax.plot(sol.t, sol.L * 100, color=color, linewidth=2.5, label=f"{name} (β={sol.params.beta:.3f})")

# Mark shock period
ax.axvspan(30, 30 + shock.duration, alpha=0.15, color='red', label='Climate Shock')

ax.set_xlabel('Days', fontsize=12)
ax.set_ylabel('Labor Force Participation (%)', fontsize=12)
ax.set_title(r'Labor Recovery by EJ Profile: $\frac{dL}{dt} = rL(1 - L/K) - \beta(EJ)$', fontsize=14)
ax.legend(loc='lower right', fontsize=10)
ax.set_xlim(0, 365)
ax.set_ylim(50, 100)

plt.tight_layout()
plt.show()

In [ ]:
# Compare recovery metrics
print("📊 Recovery Metrics by EJ Profile:\n")
print(f"{'Tract':<18} {'β':<8} {'Min LF':<10} {'Recovery':<12} {'Equilibrium':<12} {'Resilience'}")
print("-" * 75)

for (tract_id, sol), name in zip(solutions.items(), tract_names):
    recovery_str = f"{sol.recovery_time:.0f} days" if sol.recovery_time else "N/A"
    print(f"{name:<18} {sol.params.beta:.3f}    {sol.min_labor_force:.1%}      {recovery_str:<12} {sol.equilibrium:.1%}        {sol.resilience_score:.2f}")

### Sensitivity Analysis

In [ ]:
# Analyze sensitivity to parameters
ode_clean = ResilienceODE(ODEParameters(r=0.1, K=1.0, beta=0.05))

sensitivity = ode_clean.sensitivity_analysis(
    L0=0.9,
    t_span=(0, 180),
    param_ranges={
        'r': (0.02, 0.20),
        'K': (0.7, 1.0),
        'beta': (0.01, 0.30),
    },
    n_samples=15
)

# Plot sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

param_names = {'r': 'Recovery Rate (r)', 'K': 'Carrying Capacity (K)', 'beta': 'EJ Friction (β)'}
colors_sens = ['#27ae60', '#3498db', '#e74c3c']

for ax, (param, results), color in zip(axes, sensitivity.items(), colors_sens):
    values = [r[0] for r in results]
    equilibria = [r[1] for r in results]
    ax.plot(values, np.array(equilibria) * 100, color=color, linewidth=2.5, marker='o')
    ax.set_xlabel(param_names[param], fontsize=11)
    ax.set_ylabel('Equilibrium Labor Force (%)', fontsize=11)
    ax.set_title(f'Sensitivity to {param}', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.suptitle('ODE Parameter Sensitivity Analysis', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Part 3: Coupled Markov-ODE Model

Combine discrete state transitions with continuous labor force dynamics.

In [ ]:
# Create coupled model
markov = MarkovTransitionMatrix()
ode = ResilienceODE(ODEParameters(r=0.08, K=1.0, beta=0.04))

coupled = CoupledMarkovODE(markov, ode)

# Run coupled simulation
result = coupled.simulate_coupled(
    initial_distribution=np.array([0.35, 0.50, 0.08, 0.07]),
    initial_labor_force=0.92,
    n_days=365,
    shock_day=60,
    shock_severity=0.6
)

print("Coupled simulation complete!")
print(f"  Markov history shape: {result['markov_history'].shape}")
print(f"  Labor force shape: {result['labor_force'].shape}")
print(f"  Combined shape: {result['combined'].shape}")

In [ ]:
# Plot coupled results
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

days = np.arange(366)

# Plot 1: Total labor force (ODE)
ax1 = axes[0]
ax1.plot(days, result['labor_force'] * 100, color='#2c3e50', linewidth=2.5)
ax1.axvspan(60, 74, alpha=0.2, color='red', label='Climate Shock')
ax1.set_ylabel('Total Labor Force (%)', fontsize=12)
ax1.set_title('Total Labor Force (ODE)', fontsize=13)
ax1.legend()
ax1.set_xlim(0, 365)

# Plot 2: Combined (actual workers per state)
ax2 = axes[1]
colors = ['#e74c3c', '#27ae60', '#f39c12', '#3498db']
labels = ['Coastal Workers', 'Inland Workers', 'Unemployed', 'Transitioning']

ax2.stackplot(days, result['combined'].T * 100, colors=colors, labels=labels, alpha=0.8)
ax2.axvspan(60, 74, alpha=0.3, color='red')
ax2.set_xlabel('Days', fontsize=12)
ax2.set_ylabel('Workers (%)', fontsize=12)
ax2.set_title('Workforce Distribution (Coupled Markov × ODE)', fontsize=13)
ax2.legend(loc='upper right')
ax2.set_xlim(0, 365)

plt.tight_layout()
plt.show()

## Key Insights

### Markov Chain Findings
1. **Steady-state** shows long-term labor distribution under normal conditions
2. **Climate shocks** increase coastal→unemployed transition probability
3. **Mixing time** indicates how quickly the market reaches equilibrium

### ODE Findings  
1. **Environmental Justice friction (β)** significantly affects recovery speed
2. **High-burden communities** recover slower and reach lower equilibrium
3. **Recovery time** can be 2-4x longer in extreme EJ burden areas

### Policy Implications
- Target recovery resources to high-β neighborhoods
- Invest in job transition programs (reduce TRANSITIONING duration)
- Climate-proof coastal industries to reduce shock impact

In [ ]:
# Summary statistics
ode_sol = result['ode_solution']

print("📊 Simulation Summary:")
print(f"\n  Initial Labor Force: {result['labor_force'][0]:.1%}")
print(f"  Final Labor Force: {result['labor_force'][-1]:.1%}")
print(f"  Minimum (during shock): {ode_sol.min_labor_force:.1%}")
if ode_sol.recovery_time:
    print(f"  Recovery Time: {ode_sol.recovery_time:.0f} days")
print(f"  Resilience Score: {ode_sol.resilience_score:.2f}")

print("\n  Final Distribution:")
final_combined = result['combined'][-1]
for i, state in enumerate(LaborState):
    print(f"    {state.value:15s}: {final_combined[i]:.1%}")